<a href="https://colab.research.google.com/github/Abdul-Aziz-b/Pashto_spoken_digits/blob/main/Copy_of_Pashto_Spoken_Digits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Abdul-Aziz-b/Pashto_spoken_digits.git

Cloning into 'Pashto_spoken_digits'...
remote: Enumerating objects: 841, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 841 (delta 0), reused 3 (delta 0), pack-reused 838 (from 1)
Receiving objects: 100% (841/841), 45.06 MiB | 15.58 MiB/s, done.
Resolving deltas: 100% (10/10), done.
Updating files: 100% (1568/1568), done.


In [2]:
!pip install keras

In [4]:
# Step 1: Data Preparation
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.io.wavfile as wav
from keras.preprocessing import image
from keras.utils import to_categorical

def wav_to_spectrogram(audio_path, save_path, spectrogram_dimensions=(64, 64), noverlap=16, cmap='gray_r'):
    sample_rate, samples = wav.read(audio_path)
    if len(samples.shape) == 2:
        samples = np.mean(samples, axis=1)

    fig = plt.figure(figsize=(spectrogram_dimensions[0] / 100, spectrogram_dimensions[1] / 100))
    ax = plt.Axes(fig, [0, 0, 1, 1])
    ax.set_axis_off()
    fig.add_axes(ax)

    ax.specgram(samples, NFFT=256, Fs=sample_rate, noverlap=noverlap, cmap=cmap)
    fig.savefig(save_path, bbox_inches="tight", pad_inches=0)
    plt.close(fig)

def dir_to_spectrogram(audio_dir, spectrogram_dir):
    file_names = [f for f in os.listdir(audio_dir) if f.endswith('.wav')]
    for file_name in file_names:
        audio_path = os.path.join(audio_dir, file_name)
        spectrogram_path = os.path.join(spectrogram_dir, file_name.replace('.wav', '.png'))
        wav_to_spectrogram(audio_path, spectrogram_path)

audio_folder = "Recordings/"
spectrogram_folder = "spectrograms/"
dir_to_spectrogram(audio_folder, spectrogram_folder)


<ipython-input-4-81b5f2f8f3e3>:10: WavFileWarning: Chunk (non-data) not understood, skipping it.
  sample_rate, samples = wav.read(audio_path)


In [5]:
# Step 2: Load and Prepare Image Data
def prepare_data(images_dir):
    data = []
    labels = []
    for file in os.listdir(images_dir):
        label = file.split('_')[0]
        img = image.load_img(os.path.join(images_dir, file), target_size=(64, 64))
        data.append(image.img_to_array(img))
        labels.append(label)

    X = np.array(data, dtype='float32') / 255.0  # Normalize images
    y = np.array(labels)
    return X, y

X, y = prepare_data(spectrogram_folder)


In [6]:
# Step 3: Label Encoding
from sklearn.preprocessing import LabelEncoder

def encode_labels(y):
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    return to_categorical(y_encoded, num_classes=9)

y_encoded = encode_labels(y)


In [7]:
# Step 4: Define the CNN Model
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization

def voice_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(9, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

input_shape = (64, 64, 3)
model0 = voice_cnn_model(input_shape)
model0.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 62, 62, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 62, 62, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 31, 31, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 29, 29, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 29, 29, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 12, 12, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 12, 12, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 6, 6, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 4608)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       1,179,904 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 9)                   │           2,313 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,277,385 (4.87 MB)

 Trainable params: 1,276,425 (4.87 MB)

 Non-trainable params: 960 (3.75 KB)

In [16]:
# Step 5: Train the Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Define callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),  # Monitor validation loss
    ModelCheckpoint('best_model.keras', save_best_only=True)  # Save best model
]

# Train the model
history = model0.fit(X_train, y_train,
                     batch_size=50,
                     validation_data=(X_val, y_val),
                     epochs=100,
                     verbose=1,
                     callbacks=callbacks)

# Evaluate the model
results = model0.evaluate(X_val, y_val)
print(f"Validation Loss: {results[0]}, Validation Accuracy: {results[1]}")


Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.8664 - loss: 0.4858 - val_accuracy: 0.8725 - val_loss: 0.6660
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8767 - loss: 0.4094 - val_accuracy: 0.6577 - val_loss: 1.0995
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8619 - loss: 0.3900 - val_accuracy: 0.7248 - val_loss: 0.9870
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9046 - loss: 0.3174 - val_accuracy: 0.8322 - val_loss: 0.7598
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9373 - loss: 0.2373 - val_accuracy: 0.7517 - val_loss: 0.9679
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9320 - loss: 0.2326 - val_accuracy: 0.8859 - val_loss: 0.5159
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9411 - loss: 0.2115 - val_accuracy: 0.8188 - val_loss: 0.6909
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9543 - loss: 0.1580 - val_accuracy: 0.

In [17]:
# Step 6: Evaluate the Model
results = model0.evaluate(X_val, y_val)
print(f'Test Loss: {results[0]}, Test Accuracy: {results[1]}')


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9073 - loss: 0.3756 
Test Loss: 0.4161459803581238, Test Accuracy: 0.8859060406684875


In [18]:
!pip install pyyaml h5py  # Required to save models in HDF5 format
model0.save("spoken_digit_recognition_.h5")